# Spam Detection
## Author: Ventsislav Yordanov

<img src="https://media.giphy.com/media/2j5RA3SioKdck/giphy.gif" style="height:400px"/>
Image Source: https://media.giphy.com/media/2j5RA3SioKdck/giphy.gif

## Loading the needed libraries

In [1]:
import numpy as np
import pandas as pd

# Preprocessing
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Feature extraction, model evaluation and hyperparemter optimization
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Business Problem
Can we use this dataset to build a protection model that will accurately classify which messages are spam? This application is widely used from the email service providers like Gmail, Yahoo, and so on.

<img src="https://i.gifer.com/Ou1t.gif" style="height:400px"/>
Image Source: https://i.gifer.com/Ou1t.gif

## Dataset Information
The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged according being ham (legitimate) or spam.

Source: https://www.kaggle.com/uciml/sms-spam-collection-dataset/home

## Loading the Dataset

In [2]:
df = pd.read_csv("spam.csv", encoding = "latin-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Cleaning the Dataset
We can see that we have 5 columns with very confusing names. However, it's easy to see that the first column contains the target. The second one contains the message text. The other columns may be some additional notes. Let's explore them a little bit.

In [3]:
print(sum(df.iloc[:, 2].notna()))
df.iloc[:, 2].unique()

50


array([nan, ' PO Box 5249',
       ' the person is definitely special for u..... But if the person is so special',
       ' HOWU DOIN? FOUNDURSELF A JOBYET SAUSAGE?LOVE JEN XXX\\""',
       ' wanted to say hi. HI!!!\\" Stop? Send STOP to 62468"',
       'this wont even start........ Datz confidence.."', 'GN',
       '.;-):-D"',
       'just been in bedbut mite go 2 thepub l8tr if uwana mt up?loads a luv Jenxxx.\\""',
       ' bt not his girlfrnd... G o o d n i g h t . . .@"',
       ' I\'ll come up"',
       ' don\'t miss ur best life for anything... Gud nyt..."',
       ' just as a shop has to give a guarantee on what they sell. B. G."',
       ' But at d end my love compromised me for everything:-(\\".. Gud mornin:-)"',
       ' the toughest is acting Happy with all unspoken pain inside..\\""',
       ' smoke hella weed\\""', '\\" not \\"what i need to do.\\""',
       'JUST GOT PAYED2DAY & I HAVBEEN GIVEN Aå£50 PAY RISE 4MY WORK & HAVEBEEN MADE PRESCHOOLCO-ORDINATOR 2I AM FEELINGOOD

In [4]:
print(sum(df.iloc[:, 3].notna()))
df.iloc[:, 3].unique()

12


array([nan, ' MK17 92H. 450Ppw 16"', ' why to miss them', 'GE',
       'U NO THECD ISV.IMPORTANT TOME 4 2MORO\\""',
       'i wil tolerat.bcs ur my someone..... But',
       ' ILLSPEAK 2 U2MORO WEN IM NOT ASLEEP...\\""',
       'whoever is the KING\\"!... Gud nyt"', ' TX 4 FONIN HON',
       ' \\"OH No! COMPETITION\\". Who knew', 'IåÕL CALL U\\""'],
      dtype=object)

In [5]:
print(sum(df.iloc[:, 4].notna()))
df.iloc[:, 4].unique()

6


array([nan, ' just Keep-in-touch\\" gdeve.."', 'GNT:-)"',
       ' Never comfort me with a lie\\" gud ni8 and sweet dreams"',
       ' CALL 2MWEN IM BK FRMCLOUD 9! J X\\""',
       ' one day these two will become FREINDS FOREVER!"'], dtype=object)

Well, it seems that these column contains some additional comments about the messages. However, they contain only a few values and there is no documentation about them in the source. I think it's safe to remove them and try to build our machine learning model only on the message text.

In [6]:
df = df[["v1", "v2"]]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


The columns are still with confusing names, let's rename.

In [7]:
df.columns = ["class", "message"]
df.head()

,class,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Cleaning the messages
We want to keep only the important and useful words. To achieve these we will follow the steps:
1. **Keep only the words** in the message
2. Transform all words in **lower case**. We want **"Love"** and **"love"** to mean the same thing.
3. Remove all **stop words**. Stop words usually refers to the most common words in a language, for example: **"the", "a", "is", etc.** We don't need these words. They don't give us any insight.
4. Perform **stemming**. Stemming is a process in which we get the **root of the words**. We want all the different versions of the same word to be presented in one word. They all mean the same thing. Example: **"love", "loving", "lovely".**

In [8]:
# Download the last available version of the stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\venci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def clean_message(message):
    """
    Receives a raw message and clean it using the following steps:
    1. Remove all non-words in the message
    2. Transform the message in lower case
    3. Remove all stop words
    4. Perform stemming

    Args:
        message: the raw message
    Returns:
        a clean message using the mentioned steps above.
    """
    
    message = re.sub("[^A-Za-z]", " ", message)
    message = message.lower()
    message = message.split()
    stemmer = PorterStemmer()
    message = [stemmer.stem(word) for word in message if word not in set(stopwords.words("english"))]
    message = " ".join(message)
    return message

In [10]:
# Testing how our function works
message = df.message[0]
print(message)

message = clean_message(message)
print(message)

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
go jurong point crazi avail bugi n great world la e buffet cine got amor wat


In [11]:
corpus = []
for i in range(0, len(df)):
    message = clean_message(df.message[i])
    corpus.append(message)

In [12]:
corpus[:5]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though']

## Exploring the Data
Let's see what part of the messages are **spam** and what are legitimate (**ham**).

In [13]:
print(round(sum(df["class"] == "ham") / len(df) * 100, 2))
print(round(sum(df["class"] == "spam") / len(df) * 100, 2))

86.59
13.41


## Modelling the Data

### Creating a Bag of Words Model
Bag of Words model is a very popular **NLP model** used to **preprocess the texts** to classify before fitting the classification algorithms.

In [14]:
count_vectorizer = CountVectorizer()
features = count_vectorizer.fit_transform(corpus).toarray()
features.shape

(5572, 6221)

In [15]:
labels = df["class"].values
labels[:5]

array(['ham', 'ham', 'spam', 'ham', 'ham'], dtype=object)

### Splitting the Data into Test and Training Sets

In [16]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, 
    test_size = 0.20, stratify = labels, random_state = 42)

In [17]:
print(count_vectorizer.get_feature_names()[:10])
print(count_vectorizer.get_feature_names()[-10:])

['aa', 'aah', 'aaniy', 'aaooooright', 'aathi', 'ab', 'abbey', 'abdomen', 'abeg', 'abel']
['zero', 'zf', 'zhong', 'zindgi', 'zoe', 'zogtoriu', 'zoom', 'zouk', 'zs', 'zyada']


### Fitting a Multinomial Naive Bayes Classifier.
The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work. That's why we're going to try this model first.

In [18]:
nb_classifier = MultinomialNB()

In [19]:
k_fold = StratifiedKFold(n_splits = 10)
scores = cross_val_score(nb_classifier, features_train, labels_train, cv = k_fold)
print("mean:" , scores.mean(), "std:", scores.std())

mean: 0.9780143003889666 std: 0.010891013474211515


In [20]:
nb_classifier.fit(features_train, labels_train)
labels_predicted = nb_classifier.predict(features_test)
accuracy_score(labels_test, labels_predicted)

0.9775784753363229

In [21]:
confusion_matrix(labels_test, labels_predicted, labels = ["ham", "spam"])

array([[950,  16],
       [  9, 140]], dtype=int64)

## Model Selection and Improvement

### Fine-Tuning Multinomial Naive Bayes

In [22]:
kfold = StratifiedKFold(n_splits = 10)
parameters = {"alpha": np.arange(0, 1, 0.1)}
searcher = GridSearchCV(MultinomialNB(), param_grid = parameters, cv = kfold)
searcher.fit(features_train, labels_train)
best_multinomial_nb = searcher.best_estimator_

print("Best CV params", searcher.best_params_)
print("Best CV accuracy", searcher.best_score_)
print("Test accuracy of best grid search hypers:", searcher.score(features_test, labels_test))

C:\Users\venci\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\venci\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\venci\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\venci\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\venci\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\venci\Anaconda3\lib\s

Best CV params {'alpha': 0.8}
Best CV accuracy 0.9771146511106125
Test accuracy of best grid search hypers: 0.9775784753363229


## Warning: check me later
There is something strange here after the fine-tuning we have a little bit more bad accuracy.
**Is this the accuracy paradox?**

### Scores definitions
To choose our model we'll use the accuracy, recall, precision, and f1 score. Here are some definitions for this metrics:
* Accuracy: Overall, how often is the classifier correct?
* Recall: When it's actually yes, how often does it predict yes?
* Precision: When it predicts yes, how often is it correct?
* F1 score: can be interpreted as a weighted average of the precision and recall.

In [23]:
labels_predicted = best_multinomial_nb.predict(features_test)
print("Accuracy Score:", accuracy_score(labels_test, labels_predicted))
print(classification_report(labels_test, labels_predicted))

Accuracy Score: 0.9775784753363229
             precision    recall  f1-score   support

        ham       0.99      0.98      0.99       966
       spam       0.89      0.95      0.92       149

avg / total       0.98      0.98      0.98      1115



### Other Classifiers: Logistic Regression, Decision Tree, Random Forest

In [24]:
%%time

models = [LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier()]
for model in models:
    model.fit(features_train, labels_train)

    scores = cross_val_score(model, features_train, labels_train, cv = kfold)
    print(type(model))
    print("Mean score:" , scores.mean(), "Std:", scores.std())
    print()

    predictions = model.predict(features_test)
    accuracy_score(labels_test, predictions)

    labels_predicted = model.predict(features_test)
    print("Test Accuracy Score:", accuracy_score(labels_test, labels_predicted))
    print(classification_report(labels_test, labels_predicted))

<class 'sklearn.linear_model.logistic.LogisticRegression'>
Mean score: 0.9822738771421191 Std: 0.005258459452394159

Test Accuracy Score: 0.9829596412556054
             precision    recall  f1-score   support

        ham       0.98      1.00      0.99       966
       spam       1.00      0.87      0.93       149

avg / total       0.98      0.98      0.98      1115

<class 'sklearn.tree.tree.DecisionTreeClassifier'>
Mean score: 0.9744228142588096 Std: 0.009202399326537672

Test Accuracy Score: 0.9739910313901345
             precision    recall  f1-score   support

        ham       0.98      0.99      0.99       966
       spam       0.95      0.85      0.90       149

avg / total       0.97      0.97      0.97      1115

<class 'sklearn.ensemble.forest.RandomForestClassifier'>
Mean score: 0.9733027434649845 Std: 0.007789228885206418

Test Accuracy Score: 0.9668161434977578
             precision    recall  f1-score   support

        ham       0.96      1.00      0.98       966
  

## Conclusions
Well, it is controversial which is the best model. It depends on what's important for our spam detection. Personally, I think that the precision metric for the spam class is very important, but the recall is also important. In such a case when we don't know which classifier to choose. We can use the best f1 score. If some classifiers have exactly the same f1 score, we can choose the simpler one. So, if we follow this rule, we can see that the logistic regression give us the best score.

# Resources:
* https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/
* https://en.wikipedia.org/wiki/Natural_language_processing
* https://en.wikipedia.org/wiki/Stop_words
* https://en.wikipedia.org/wiki/Stemming
* http://scikit-learn.org/stable/index.html
* https://stats.stackexchange.com/questions/250273/benefits-of-stratified-vs-random-sampling-for-generating-training-data-in-classi/250742#250742
* https://stats.stackexchange.com/questions/117643/why-use-stratified-cross-validation-why-does-this-not-damage-variance-related-b/117649#117649?newreg=2a9d984517504dcbbf55fda2f11489b7
* https://stats.stackexchange.com/questions/49540/understanding-stratified-cross-validation

# TODOs
* Try different values for test_size in the "train_test_split" function
* Try CountVectorizer with some values for the "max_features" parameter
* Use pickle to save the trained classifiers

# Future Ideas
* Compare more classifiers
* Try to use TFIDF and compare the results
* Try to use Dimensionality Reduction

# Notes
* Reduce the number of the features, because they are too many now and this may leads to overfitting
* Add more data to avoid high bias